In [15]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry 
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display
import assignment_2_2023.msg
from assignment_2_2023.msg import PlanningAction
from assignment_2_2023.msg import Info
import actionlib
import actionlib.msg
from matplotlib.widgets import TextBox


In [16]:
def on_odom(msg):
    global info, pub_info, hbox_1, hbox_2, info_old
    pos = msg.pose.pose.position
	
	# Get the current velocity
    vel_linear_x = msg.twist.twist.linear.x
    vel_angular_z = msg.twist.twist.angular.z
    
    # rospy.loginfo("Velocity: %f, %f", vel_linear_x, vel_angular_z)
    
    # Prepare and publish the info message
    info = Info()
    info.x = pos.x
    info.y = pos.y
    info.vel_linear_x = vel_linear_x
    info.vel_angular_z = vel_angular_z
    pub_info.publish(info)
    if(abs(info_old.x-info.x) > 0.1 or abs(info_old.y-info.y) > 0.1):
        hbox_1.children = ([widgets.Label("{:.1f}".format(info.x)),])
        hbox_2.children = ([widgets.Label("{:.1f}".format(info.y)),])
        info_old = info

In [18]:
global client, pub_info, info_old
info_old = Info()
info_old.x = 0.0
info_old.y = 0.0
hbox_1 = widgets.HBox()
hbox_2 = widgets.HBox()

rospy.init_node('client_node')

# sub_odom is a subscriber that receives Odometry data
sub_odom = rospy.Subscriber("/odom", Odometry, on_odom)

# info_pub is a publisher that sends the info (x,y,vel_linear_x, vel_angular_z)
pub_info = rospy.Publisher('/info_pos_vel', Info, queue_size = 1)

client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()



True

In [17]:

def on_coordinates_sent(b):
    global x_coordinate, y_coordinate, client, info, vbox, html_code_vertical_shorter
    if(b.description == "Cancel"):
        b.description = "Send Coordinates"
        client.cancel_goal()
        
        
        hbox = widgets.HBox()
        hbox_try0 = widgets.HBox((widgets.Label("Deleted"),))
        hbox_try1 = widgets.HBox((widgets.Label("{:.1f}".format(info.x)),))
        hbox_try2 = widgets.HBox((widgets.Label("{:.1f}".format(info.y)),))
        hbox_try0.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try1.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try2.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')


        hbox.children = ([hbox_try0, widgets.HTML(html_code_vertical_shorter), hbox_try1, widgets.HTML(html_code_vertical_shorter), hbox_try2])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
    elif(b.description == "Send Coordinates"):
        goal = assignment_2_2023.msg.PlanningGoal()
        goal.target_pose.pose.position.x = x_coordinate.value
        goal.target_pose.pose.position.y = y_coordinate.value
        
        hbox = widgets.HBox()
        hbox_try0 = widgets.HBox((widgets.Label("Start"),))
        hbox_try1 = widgets.HBox((widgets.Label(str(x_coordinate.value)),))
        hbox_try2 = widgets.HBox((widgets.Label(str(y_coordinate.value)),))
        hbox_try0.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try1.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try2.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')

        hbox.children = ([hbox_try0, widgets.HTML(html_code_vertical_shorter), hbox_try1, widgets.HTML(html_code_vertical_shorter), hbox_try2])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
        rospy.loginfo("Goal set to x = %f, y = %f", goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        client.send_goal(goal)
        print("Coordinates sent with x = %.1f, y = %.1f" % (x_coordinate.value, y_coordinate.value))
        b.description = "Cancel"

In [22]:
global x_coordinate, y_coordinate, send_coordinates, vbox, hbox_1, hbox_2, html_code_vertical_shorter

x_coordinate = widgets.FloatSlider( 
value=0.0,
min=-9.0, 
max=9.0, 
step=0.1,
description='X:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
y_coordinate = widgets.FloatSlider( 
value=1.0,
min= -9.0, 
max=9.0, 
step=0.1,
description='Y:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
display(x_coordinate, y_coordinate)

send_coordinates = widgets.Button(description="Send Coordinates", layout=widgets.Layout(width='50%', height='60px', color='black', font_weight='bold'))
send_coordinates.style.button_color = 'orange'
send_coordinates.style.font_weight = 'bold'
output = widgets.Output()
display(send_coordinates, output)
send_coordinates.on_click(on_coordinates_sent)

## Show interactive robot position
hbox_coordinates = widgets.HBox()
hbox_0 = widgets.HBox((widgets.Label("Coordinates:"),))
hbox_0.layout = widgets.Layout(width='33%', height='60px', overflow_y='hidden',justify_content='center', align_items='center')
hbox_1 = widgets.HBox((widgets.Label("{:.1f}".format(0.0)),))
hbox_1.layout = widgets.Layout(width='33%', height='60px', overflow_y='hidden', justify_content='center', align_items='center')
hbox_2 = widgets.HBox((widgets.Label("{:.1f}".format(1.0)),))
hbox_2.layout = widgets.Layout(width='33%', height='60px', overflow_y='hidden', justify_content='center', align_items='center')
html_code_vertical = '<div style="border-left: 1px solid #000; height: 100px;"></div>'
html_code_vertical_shorter = '<div style="border-left: 1px solid #000; height: 20px;"></div>'

html_code_horizontal = '<hr style="border-top: 1px solid #000;">'


hbox_coordinates.layout = widgets.Layout(width='100%', height='60px', overflow_y='hidden', )

vbox_coordinates = widgets.VBox()
vbox_coordinates.layout = widgets.Layout(width='100%', height='100px', overflow_y='hidden')
# hbox_coordinates_0 = widgets.HBox([hbox_0, hbox_1, hbox_2])
hbox_coordinates.children = [widgets.HTML(html_code_vertical), hbox_0, widgets.HTML(html_code_vertical), hbox_1, widgets.HTML(html_code_vertical), hbox_2, widgets.HTML(html_code_vertical)]
vbox_coordinates.children = [widgets.HTML(html_code_horizontal), hbox_coordinates, widgets.HTML(html_code_horizontal)]
# display(widgets.HTML(html_code_horizontal))
# display(hbox_coordinates)
#textbox = TextBox('Text:', initial="Initial Text", color='gray', hovercolor='cyan')
display(vbox_coordinates)
## Show table with coordinates
# Display the tables
vbox = widgets.VBox()
hbox = widgets.HBox()
hbox_try0 = widgets.HBox((widgets.Label("Operation"),))
hbox_try1 = widgets.HBox((widgets.Label("X"),))
hbox_try2 = widgets.HBox((widgets.Label("Y"),))
hbox_try0.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
hbox_try1.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
hbox_try2.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')

hbox.children = ([hbox_try0, widgets.HTML(html_code_vertical_shorter), hbox_try1, widgets.HTML(html_code_vertical_shorter), hbox_try2])
vbox.children = tuple([hbox,])
display(vbox)



FloatSlider(value=0.0, continuous_update=False, description='X:', max=9.0, min=-9.0)

FloatSlider(value=1.0, continuous_update=False, description='Y:', max=9.0, min=-9.0)

Button(description='Cancel', layout=Layout(height='60px', width='50%'), style=ButtonStyle(button_color='orange…

Output()

[INFO] [1712050825.754560, 5767.706000]: Goal set to x = 0.000000, y = 1.000000
Coordinates sent with x = 0.0, y = 1.0
[INFO] [1712050835.623813, 5774.723000]: Goal set to x = 0.000000, y = -2.200000
Coordinates sent with x = 0.0, y = -2.2
